## Fig 3: Halo model power spectra 

In [ ]:
# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt

# Third-party imports
import camb # Calculates the linear matter power spectrum

# Halo Model library imports
sys.path.append('../src/')
import halomodel as halo

In [ ]:
# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
As = 1.97448e-9
ns = 0.96
w = -1.0
wa = 0.0
m_nu = 0.0 # in eV
sigma_8_set = True # if true uses the following value
sigma_8_in  = 0.8

# Colours
col_lin = 'black'
col_mat = 'C0'
col_gal = 'C1'
col_mg  = 'C4'

# Line styles
ls_li = '-'
ls_hm = '-'
ls_2h = '--'
ls_1h = ':'

# Labels
klab = r'$k\,/\,h \mathrm{Mpc}^{-1}$'

In [ ]:
# k range [h/Mpc]
kmin = 1e-3; kmax = 10.
nk = 101
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

# Redshift
z = 0.

# CAMB parameters
zmax_CAMB = 2.
kmax_CAMB = 200.

In [ ]:
# Sets cosmological parameters in camb to calculate the linear power spectrum
pars = camb.CAMBparams()
wb = Omega_b*h**2
wc = Omega_c*h**2

# This function sets standard and helium set using BBN consistency
pars.set_cosmology(ombh2=wb, omch2=wc, H0=100.*h, mnu=m_nu, omk=Omega_k)
pars.set_dark_energy(w=w, wa=wa, dark_energy_model='ppf') 
pars.InitPower.set_params(As=As, ns=ns, r=0)
pars.set_matter_power(redshifts=[z], kmax=kmax_CAMB) # Setup the linear matter power spectrum

# extract parameters from CAMB
Omega_m  = pars.omegam
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[[z].index(0.)]).item()

if sigma_8_set:
    scaling = (sigma_8_in/sigma_8)**2
    As *= scaling
    pars.InitPower.set_params(As=As, ns=ns, r=0.)

Pk_lin = camb.get_matter_power_interpolator(pars, 
                                            nonlinear=False, 
                                            hubble_units=True, 
                                            k_hunit=True, 
                                            kmax=kmax_CAMB,
                                            var1=camb.model.Transfer_tot,
                                            var2=camb.model.Transfer_tot, 
                                            zmax=zmax_CAMB,
                                           )
Pks_lin = Pk_lin.P(z, ks) # Single out the linear P(k) interpolator
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[[z].index(0.)]).item()

In [ ]:
# Initialise halo model
hmod = halo.model(z, Omega_m, name='Tinker et al. (2010)', Dv=330.)

# Calculate the matter density
print('Mean matter density [log10((Msun/h)/(Mpc/h)^-3)]:', np.log10(hmod.rhom))

In [ ]:
# To speed things we use these settings if you want more precision use the above numbers
Mmin = 1e9; Mmax = 1e15; nM = 256
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# Lagrangian radii corresponding to halo masses; 
Rs = hmod.Lagrangian_radius(Ms)

In [ ]:
# Get sigma(R) from CAMB
sigmaRs = camb_results.get_sigmaR(Rs, hubble_units=True, return_R_z=False)[[z].index(z)]

In [ ]:
# Initialise profile class
rvs = hmod.virial_radius(Ms)
cs = halo.concentration(Ms, z, method='Duffy et al. (2008)', halo_definition='Mvir')
matter_profile = halo.matter_profile(ks, Ms, rvs, cs, hmod.Om_m)
print(matter_profile)

In [ ]:
# Simple (unrealistic) HOD model
def HOD(M, Mmin=1e12, Msat=1e13):
    if M < Mmin:
        return 0.
    elif Mmin <= M and M < Msat:
        return 1.
    else:
        #return np.rint(M/Msat)
        return M/Msat
HOD = np.vectorize(HOD)
 
# Compute the mean galaxy density corresponding to our HOD
Ng = HOD(Ms)
rhog = hmod.average(Ms, sigmaRs, Ng)
print('Mean galaxy density [(Mpc/h)^-3]:', rhog)

# Plot HOD
plt.loglog(Ms, Ng, color=col_gal)
plt.xlabel('Halo mass [$h^{-1}\,M_\odot$]')
plt.ylabel('Number of galaxies')
plt.ylim(bottom=1e-1)
plt.xlim((1e11, 1e16))
plt.show()

In [ ]:
# Galaxy profile
Uk = halo.window_function(ks, rvs, cs, profile='NFW')
galaxy_profile = halo.profile.Fourier(ks, Ms, Uk, amp=Ng, norm=rhog, discrete_tracer=True)
print(galaxy_profile)

In [ ]:
# Calculate the halo-model power spectrum
profiles = {'m': matter_profile, 'g': galaxy_profile}
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, profiles)

In [ ]:
# Plot matter, galaxy and cross spectra

# Axis limits
Pkmin, Pkmax = 1e1, 1e5
kmin_plot, kmax_plot = 1e-3, 1e1
rmin, rmax = 1e-1, 1e2
smin, smax = 0., 2.4

# Initialise plot
plt.subplots(3, 1, figsize=(5, 7), dpi= 100, sharex=True)
plt.subplots_adjust(wspace=0.1, hspace=0.1) 

# Lists for plots
Pks = [Pk_2h['m-m'], Pk_1h['m-m'], Pk_hm['m-m'], Pk_hm['g-g'], Pk_hm['m-g']]
cols = 3*[col_mat]+[col_gal, col_mg]
lss = [ls_2h, ls_1h, ls_hm, ls_hm, ls_hm]
labs = [None, None, 'matter', 'galaxy', 'matter-galaxy']

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, ls=ls_li, label='linear')
for (ls, lab) in zip([ls_2h, ls_1h], ['2-halo term', '1-halo term']):
    plt.plot(np.nan, ls=ls, label=lab, color='black')
for (Pk, col, ls, lab) in zip(Pks, cols, lss, labs):
    plt.loglog(ks, Pk, color=col, ls=ls, label=lab)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(ncol=2, loc='lower left', fontsize='9')

# Lists for plots
Pks = [Pk_2h['m-m'], Pk_1h['m-m'], Pk_hm['m-m'],
       Pk_2h['g-g'], Pk_1h['g-g'], Pk_hm['g-g'],
       Pk_2h['m-g'], Pk_1h['m-g'], Pk_hm['m-g']]
cols = 3*[col_mat]+3*[col_gal]+3*[col_mg]
lss = 3*[ls_2h, ls_1h, ls_hm]

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.loglog(ks, Pk/Pks_lin, color=col, ls=ls)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P^\mathrm{lin}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_hm['m-m'], color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.semilogx(ks, Pk/Pk_hm['m-m'], color=col, ls=ls)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P_\mathrm{mm}(k)$')
plt.ylim((smin, smax))

# Finish
plt.savefig('plots/power_HOD_all.pdf', bbox_inches='tight')
plt.show()